In [1]:

"""

Date: 21 Dec 2019

Python version:      3.7
Tensorboard version: 1.14.0
PyTorch version:     1.2.0

@author: Maksim Lavrov

Transfer learning on CIFAR10 dataset
    • With a flexible layer implemented instead of the last convolutional layer

"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms

In [2]:
# Loading data
# transforms
transform = transforms.Compose(
    [transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    #transforms.Lambda(torgb) # Transform grayscale to RGB
    ])

# datasets
trainset = torchvision.datasets.CIFAR10('./data',
    download=True,
    train=True,
    transform=transform)
testset = torchvision.datasets.CIFAR10('./data',
    download=True,
    train=False,
    transform=transform)

#create a dataset subset to reduce training time

sampler_train = list(range(0, len(trainset), 5000))
sampler_test = list(range(0, len(testset), 5000))
trainset_samp = torch.utils.data.Subset(trainset, sampler_train)
testset_samp = torch.utils.data.Subset(testset, sampler_test)

#set size of batch and learning rate
batch_size=4
lr=0.001

# dataloaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                        shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                        shuffle=False, num_workers=2)

# constant for classes
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

n_classes = 10

Files already downloaded and verified
Files already downloaded and verified


class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args

    def forward(self, x):
        return x.view(self.shape)

class FlexiLayer(nn.Conv2d):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1,
                 padding=0, dilation=1, groups=1,
                 bias=True, padding_mode='zeros'):
        kernel_size = kernel_size
        stride = stride
        padding = padding
        dilation = dilation
        super(FlexiLayer, self).__init__(
            in_channels, out_channels, kernel_size, stride, padding, dilation,
            groups, bias, padding_mode)
        
        
        self.threshold1 = nn.parameter.Parameter(torch.randn((4, 9, 8, 8)), requires_grad=True)
        self.memorized = self.threshold1.clone()
        self.memorized_1loop = []
        self.memorized_1pixel = []
        self.thresh_max = []
        self.thresh_min = []
        self.thresh_mean = []
            
    def forward(self, t):
        
        t_1 = F.relu(F.conv2d(t, self.weight)) # get convolution result
        t_2 = F.avg_pool2d(t, kernel_size=5, stride=1) # get avg result with the same kernel size
        t_2 = torch.cat((t_2, t_2, t_2), 1)
        m = nn.Sigmoid()
        print(t_2.shape)
        print(self.threshold1.shape)
        condmax = torch.sub(t_2, self.threshold1)
        condconv = torch.sub(t_2, self.threshold1)
        t_2 = m(condmax*50)*t_2 # 
        print('cond', condconv.shape)
        print('t_1', t_1.shape)
        t_1 = m(condconv*(-50))*t_1 # 
        t = torch.add(t_2, t_1)
        #t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        return t

In [3]:
#Model Building
#Feature Extraction
net = models.resnet18(pretrained=True)
for param in net.parameters():
    param.requires_grad = False

In [4]:
num_ftrs = net.fc.in_features
net.fc = nn.Sequential(
    nn.Linear(num_ftrs, 120),
    #Reshape(4,3,12,12),
    #FlexiLayer(in_channels=3, out_channels=9, kernel_size=5),
    #nn.Linear(in_features= 3 * 56 * 56, out_features=120),
    nn.Linear(in_features=120, out_features=60),
    nn.Linear(in_features=60, out_features=10)
)
#net.fc.weight.requires_grad = True
#net.fc.bias.requires_grad = True

In [5]:
for name, param in net.named_parameters():
    if param.requires_grad:
        print (name, param.data)

fc.0.weight tensor([[ 0.0307, -0.0167, -0.0409,  ...,  0.0399, -0.0322,  0.0184],
        [-0.0338,  0.0321, -0.0047,  ..., -0.0241, -0.0194,  0.0076],
        [ 0.0306,  0.0022,  0.0387,  ...,  0.0348, -0.0010, -0.0372],
        ...,
        [-0.0103,  0.0039, -0.0159,  ..., -0.0340,  0.0049,  0.0179],
        [-0.0323, -0.0180,  0.0340,  ..., -0.0038, -0.0231,  0.0183],
        [ 0.0128,  0.0085, -0.0207,  ..., -0.0129,  0.0312,  0.0309]])
fc.0.bias tensor([ 0.0005, -0.0232, -0.0103,  0.0324, -0.0351,  0.0007,  0.0219, -0.0174,
         0.0080, -0.0348,  0.0169,  0.0415,  0.0217,  0.0103, -0.0186, -0.0226,
         0.0009, -0.0425, -0.0398,  0.0055,  0.0436, -0.0341,  0.0120,  0.0359,
         0.0432,  0.0313,  0.0305,  0.0239,  0.0439, -0.0269,  0.0202, -0.0073,
        -0.0215,  0.0041,  0.0420, -0.0002, -0.0072, -0.0051, -0.0411, -0.0117,
        -0.0280,  0.0184, -0.0277,  0.0086,  0.0379, -0.0028,  0.0062, -0.0298,
        -0.0438, -0.0266, -0.0080,  0.0009, -0.0047, -0.0118,  0

In [6]:
#num_ftrs = net.fc.in_features
#net.fc = nn.Sequential(FlexiLayer(in_channels=1, out_channels=256, kernel_size=5),
#                      nn.Linear(256, n_classes))
# Add on classifier
#net.classifier[6] = nn.Sequential(
 #                     FlexiLayer(in_channels=3, out_channels=6, kernel_size=5),
  #                    nn.Linear(256, n_classes))

#num_ftrs = net.fc.in_features
#net.fc = nn.Linear(num_ftrs, n_classes) # 10 classes in the dataset
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [7]:
#helper functions

def images_to_probs(net, images):
    '''
    Generates predictions and corresponding probabilities from a trained
    network and a list of images
    '''
    output = net(images)
    # convert output probabilities to predicted class
    _, preds_tensor = torch.max(output, 1)
    preds = np.squeeze(preds_tensor.numpy())
    return preds, [F.softmax(el, dim=0)[i].item() for i, el in zip(preds, output)]


def plot_classes_preds(net, images, labels):
    '''
    Generates matplotlib Figure using a trained network, along with images
    and labels from a batch, that shows the network's top prediction along
    with its probability, alongside the actual label, coloring this
    information based on whether the prediction was correct or not.
    Uses the "images_to_probs" function.
    '''
    preds, probs = images_to_probs(net, images)
    # plot the images in the batch, along with predicted and true labels
    fig = plt.figure(figsize=(12, 48))
    for idx in np.arange(4):
        ax = fig.add_subplot(1, 4, idx+1, xticks=[], yticks=[])
        matplotlib_imshow(images[idx], one_channel=True)
        ax.set_title("{0}, {1:.1f}%\n(label: {2})".format(
            classes[preds[idx]],
            probs[idx] * 100.0,
            classes[labels[idx]]),
                    color=("green" if preds[idx]==labels[idx].item() else "red"))
    return fig

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [8]:
running_loss = 0.0
for epoch in range(9):  # loop over the dataset multiple times
    
    total_loss = 0
    total_correct = 0

    for i, data in enumerate(trainloader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        preds = net(inputs) # Pass batch

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item() * batch_size
        total_correct += get_num_correct(preds, labels)

        running_loss += loss.item()
            
    print("epoch:", epoch, "loss:", total_loss)
        
print('Finished Training')

epoch: 0 loss: 62361.094607800245
epoch: 1 loss: 55696.136185228825
epoch: 2 loss: 54434.94817149639
epoch: 3 loss: 54116.29214271903
epoch: 4 loss: 53998.34625171125
epoch: 5 loss: 53730.587011791766
epoch: 6 loss: 53298.87064033002
epoch: 7 loss: 53311.091692097485
epoch: 8 loss: 53350.631316021085
Finished Training


In [9]:
print('Accuracy on the training dataset is ', total_correct / len(trainset))

Accuracy on the training dataset is  0.64042


In [10]:
#evaluate model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy on the test dataset is: %f %%' % (
    100 * correct / total))

Accuracy on the test dataset is: 62.820000 %


In [11]:
#save the trained model 
path = "./models/ResNet_base_20202203.pt" 
torch.save(net.state_dict(), path)

In [12]:
#15:21 start 15:25 finish